In [1]:
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import generative_adversarial as gan
import dwdii_transforms
import numpy as np
import random

Using Theano backend.


In [2]:
import theano
#theano.config.device = 'gpu'
#theano.config.floatX = 'float32'

print theano.config.device
print theano.config.floatX

cpu
float32


In [2]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"

### Load Data
In this section we load the image data and scale to 34x34.

In [3]:
maxData = 2200
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = None, verboseFreq = 200, imgResize=(34,34), theseEmotions=["happiness"])
print X_data.shape
print Y_data.shape

0.000000: Yannos_Papantoniou_0001.jpg
0.088456: Michael_Patrick_King_0002.jpg
0.176913: James_Watt_0001.jpg
(1734, 34, 34)
(1734, 1)


In [4]:
X_train, X_test, Y_train, Y_test = dwdii_transforms.trainingTestSplit(X_data, Y_data, 0.8, True)
shp = X_train.shape[1:]
print "Shape: " + str(shp)

1387
X_train: (1387, 1, 34, 34)
X_test: (347, 1, 34, 34)
Y_train: (1387, 1)
Y_test: (347, 1)
Shape: (1, 34, 34)


### Create Generative and Discriminative Models

In [5]:
genModel = gan.buildGenerativeModel(verbose=False)

In [6]:
discrimModel = gan.buildDiscriminativeModel(shape=shp, verbose=False)

### Discriminator Training

In [7]:
ntrain = 600
trainidx = random.sample(range(0,X_train.shape[0]), ntrain)
XT = X_train[trainidx,:,:,:]

In [ ]:
# Pre-train the discriminator network ...
noise_gen = np.random.uniform(0,1,size=[XT.shape[0],100])
generated_images = genModel.predict(noise_gen)
X = np.concatenate((XT, generated_images))
n = XT.shape[0]
y = np.zeros([2*n,2])
y[:n,1] = 1
y[n:,0] = 1

In [ ]:
gan.makeTrainable(discrimModel,True)
discrimModel.fit(X,y, nb_epoch=1, batch_size=128)

In [ ]:
y_hat = discrimModel.predict(X)

In [ ]:
# Measure accuracy of pre-trained discriminator network
y_hat_idx = np.argmax(y_hat,axis=1)
y_idx = np.argmax(y,axis=1)
diff = y_idx-y_hat_idx
n_tot = y.shape[0]
n_rig = (diff==0).sum()
acc = n_rig*100.0/n_tot
print "Accuracy: %0.02f pct (%d of %d) right"%(acc, n_rig, n_tot)

### Stacked GAN and Training

In [8]:
loadWeights = True
if loadWeights:
    genModel.load_weights("dwdii-genModel-03-01.hdf5")
    discrimModel.load_weights("dwdii-discrimModel-03-01.hdf5")

# Build the stacked GAN model
GAN = gan.buildStackedGanModel(genModel, discrimModel)
if loadWeights:
    GAN.load_weights("dwdii-GAN-03-01.hdf5")

In [9]:
gan.makeTrainable(discrimModel,False)

In [10]:
gan.plotGenerative(0, genModel)

In [11]:
# Train for N epochs at original learning rates
gan.train_for_n(X_train, GAN, genModel, discrimModel, nb_epoch=6000, plt_frq=50,BATCH_SIZE=32)

(149, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(149, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(149, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(149, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(149, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(149, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(249, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(249, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(249, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(249, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(249, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(249, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(349, 'Error saving in plotG

/usr/lib/pymodules/python2.7/matplotlib/pyplot.py:412: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_num_figures`).
  max_open_warning, RuntimeWarning)


(1099, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1099, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1099, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1099, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1099, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1099, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1199, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1199, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1199, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1199, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1199, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1199, 'Error saving in plotGenerative, retrying: ', <type 'exceptions.IOError'>)
(1299, 'Error sa

KeyboardInterrupt: 

In [12]:
ver = "03-02"
GAN.save_weights("dwdii-GAN-" + ver + ".hdf5", overwrite=True)
discrimModel.save_weights("dwdii-discrimModel-" + ver + ".hdf5", overwrite=True)
genModel.save_weights("dwdii-genModel-" + ver + ".hdf5", overwrite=True)